In [1]:
# pip install transformers
# pip install bitsandbytes
# pip install -qU langchain_community pypdf
# pip install -qU "langchain-chroma>=0.1.2"
# pip install -U sentence-transformers
# pip install sentence_transformers
# pip install langchain==0.0.174 -i https://pypi.doubanio.com/simple/ --trusted-host pypi.doubanio.com

from uuid import uuid4

from transformers import pipeline
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.documents import Document
from langchain_chroma import Chroma

prompt = input("Enter a prompt: ")

embeddings = HuggingFaceEmbeddings(model_name="Snowflake/snowflake-arctic-embed-m", model_kwargs = {'device': 'cuda'}, show_progress=True)

loader = PyPDFLoader(
    "./data/falcon-users-guide-2021-09-compressed.pdf",
)

document = loader.load()

document_1 = Document(
    page_content=document[0].page_content,
    id=1,
)

vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not neccesary
)

documents = [
    document_1,
]

uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

results = vector_store.similarity_search(
    prompt,
    k=1
)

context = results[0].page_content

print(results[0].page_content)
    
rag_prompt = f"You are a helpful engineer who is trying to help a user with a problem. The user has a problem statement {prompt}. The user has provided the following context: {context}. Please provide a helpful response to the user."

pipe = pipeline(model="meta-llama/Meta-Llama-3-8B", model_kwargs={"load_in_4bit": True}, device_map="auto")
output = pipe(rag_prompt, do_sample=True, top_p=0.95)

print("\n" + output[0]["generated_text"] + "\n")

2024-09-05 01:01:35.262456: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-05 01:01:35.304098: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX512F AVX512_VNNI, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/tmp/ipykernel_35625/2821227197.py:19: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and wi

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


 
© Space Exploration Technologies Corp.  All rights reserved.  
  
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
COPYRIGHT  
Subject to the existing rights of third parties, Space Exploration Technologies Corp. (SpaceX) is the owner of the copyright 
in this work, and no portion hereof is to be copied, reproduced, or disseminated without the prior written consent of 
SpaceX.  



Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
/home/ubuntu/.local/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:435: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(



You are a helpful engineer who is trying to help a user with a problem. The user has a problem statement What is Falcon 9. The user has provided the following context:  
© Space Exploration Technologies Corp.  All rights reserved.  
  
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
COPYRIGHT  
Subject to the existing rights of third parties, Space Exploration Technologies Corp. (SpaceX) is the owner of the copyright 
in this work, and no portion hereof is to be copied, reproduced, or disseminated without the prior written consent of 
SpaceX.  
. Please provide a helpful response to the user.  




In [ ]:
import gradio as gr


def echo(input_text):
    prompt = f"You are a helpful engineer who is trying to help a user with a problem. The user has a problem statement {input_text}. The user has provided the following context: {context}. Please provide a helpful response to the user."

    return pipe(prompt, do_sample=True, top_p=0.95)[0]["generated_text"]

demo = gr.Interface(
    fn=echo,
    inputs=["text"],
    outputs=["text"],
)

demo.launch()